In [1]:
import pytorch_lightning as pl
from src import DataModule, Model
import torch
from pytorch_lightning.metrics.functional.classification import accuracy
from tqdm import tqdm
import numpy as np

In [2]:
checkpoint = torch.load('resnet50-val_acc=0.86215.ckpt')
checkpoint['hyper_parameters']

{'lr': 0.0003,
 'optimizer': 'Adam',
 'batch_size': 64,
 'max_epochs': 50,
 'precision': 16,
 'subset': 0,
 'test_size': 0.2,
 'seed': 42,
 'size': 512,
 'backbone': 'resnet50'}

In [3]:
model = Model.load_from_checkpoint('resnet50-val_acc=0.86215.ckpt')
model.hparams

"backbone":   resnet50
"batch_size": 64
"lr":         0.0003
"max_epochs": 50
"optimizer":  Adam
"precision":  16
"seed":       42
"size":       512
"subset":     0
"test_size":  0.2

In [4]:
def evaluate(model, dl):   
    model.eval()
    model.cuda()
    acc = []
    with torch.no_grad():
        t = tqdm(dl)
        for x, y in t:
            x, y = x.cuda(), y.cuda()
            y_hat = model(x)
            acc.append(accuracy(y_hat, y).item())
            t.set_description(f"acc {np.mean(acc):.5f}")

In [5]:
dm = DataModule()
dm.setup()
evaluate(model, dm.val_dataloader())

  0%|          | 0/67 [00:00<?, ?it/s]

Training samples:  17117
Validation samples:  4280


acc 0.86211: 100%|██████████| 67/67 [00:47<00:00,  1.40it/s]


In [6]:
class Preprocess(torch.nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x):
        x = x.float() / 255.
        x = x.permute(0, 3, 1, 2)
        return x 
    
class Postprocess(torch.nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x):
        return torch.argmax(x, dim=1)

In [10]:
script = torch.jit.script(torch.nn.Sequential(
    Preprocess(),
    model.resnet.cpu(),
    Postprocess()
))
torch.jit.save(script, "resnet50.pt")

In [11]:
def evaluate2(model, dl):   
    model.eval()
    model.cuda()
    acc = []
    with torch.no_grad():
        t = tqdm(dl)
        for x, y in t:
            x, y = x.cuda(), y.cuda()
            # simulate test
            x *= 255. 
            x = x.permute(0, 2, 3, 1).long()
            y_hat = model(x)
            acc.append(accuracy(y_hat, y).item())
            t.set_description(f"acc {np.mean(acc):.5f}")

In [12]:
loaded = torch.jit.load('resnet50.pt')
evaluate2(loaded, dm.val_dataloader())

acc 0.86211: 100%|██████████| 67/67 [00:43<00:00,  1.52it/s]
